### Goal
- Input: RGB image.
- Teacher: Depth Anything V2 → predicts a high-quality depth map.
- Student: A smaller, faster model → learns to mimic the teacher’s depth map predictions.

In [ ]:
import torch
from torch import nn

import torchvision
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm.auto import tqdm
from timeit import default_timer as timer
import random

from SegNet import SegNet


In [ ]:
# for google colab
# from google.colab import drive
# drive.mount('/content/drive')

# !unzip /content/drive/MyDrive/data

In [ ]:
class DepthDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, depth_maps_dir, transform=None):
        self.images_dir = images_dir
        self.depth_maps_dir = depth_maps_dir
        self.transform = transform
        
        self.files = [f for f in os.listdir(images_dir) if f.lower().endswith('.png')]
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.files[idx])
        depth_map_path = os.path.join(self.depth_maps_dir, self.files[idx])
        
        image = plt.imread(image_path)
        depth_map = plt.imread(depth_map_path)[:,:,0]
        
        if self.transform:
            image = self.transform(image)
            depth_map = self.transform(depth_map)

        return image, depth_map

In [ ]:
dataset = DepthDataset(
    images_dir='data/depth_data/images',
    depth_maps_dir='data/depth_data/depth_maps',
    transform=ToTensor(),
)
print(dataset.__len__(), "images and depth maps found in dataset.")

# Create a figure with 2 rows and 5 columns
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Get 5 random samples from the dataset
indices = np.random.choice(len(dataset), 5, replace=False)

for i, idx in enumerate(indices):
    # Get image and depth map
    image, depth_map = dataset[idx]
    
    # Convert tensors back to numpy for plotting
    if isinstance(image, torch.Tensor):
        image = image.permute(1, 2, 0).numpy()
    if isinstance(depth_map, torch.Tensor):
        depth_map = depth_map.numpy().squeeze()
    
    # Plot RGB image in first row
    axes[0, i].imshow(image)
    axes[0, i].set_title(f'Image {idx}')
    axes[0, i].axis('off')
    
    # Plot depth map in second row
    axes[1, i].imshow(depth_map, cmap='gray')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
torch.manual_seed(42)
train, val, test = torch.utils.data.random_split(dataset, [0.7, 0.15, 0.15])
train.__len__(), val.__len__(), test.__len__()

In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)

images, depth_maps = next(iter(train_loader))
print(f'Batch of images shape: {images.shape}')
print(f'Batch of depth maps shape: {depth_maps.shape}')

Sorce: https://arxiv.org/pdf/1406.2283

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SegNet(in_channels=3, out_channels=1).to(device)

In [ ]:
# Replace CrossEntropyLoss with MSELoss for depth estimation
loss_fn = nn.MSELoss()  # Use MSE for regression tasks
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Fix the training loop - pass both prediction and target to loss function
torch.manual_seed(42)  
train_time_start_on_cpu = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
    print(f'Epoch: {epoch + 1} \n------------')
    # Training loop
    train_loss = 0.0
    for batch, (X, y) in enumerate(train_loader):
        # Move data to device and ensure float32
        X = X.to(device).float()
        y = y.to(device).float()
        
        model.train()
        y_pred = model(X)
        
        # Fix: Pass both prediction and target to loss function
        loss = loss_fn(y_pred, y) 
        train_loss += loss.item()  # Use .item() to get scalar value
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 40 == 0:
            print(f'Loss: {loss.item():.5f} | Batch: {batch + 1}/{len(train_loader)}')
       
    train_loss /= len(train_loader) 
    
    # Testing loop
    test_loss, test_acc = 0.0, 0.0
    model.eval()
    with torch.inference_mode():
        for X_test, y_test in test_loader:
            # Move test data to device
            X_test = X_test.to(device).float()
            y_test = y_test.to(device).float()
            
            y_pred = model(X_test)
            loss = loss_fn(y_pred, y_test)  # Fix: Pass both arguments
            test_loss += loss.item()  # Use .item() here too
            test_acc += (y_pred.squeeze() - y_test).abs().mean().item()

        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        
    print(f'Train Loss: {train_loss:.5f} | Test Loss: {test_loss:.5f} | Test MAE: {test_acc:.5f}')
    print('-----------------------------------')
    
train_time_end_on_cpu = timer()
total_train_time = train_time_end_on_cpu - train_time_start_on_cpu
print(f'Total training time: {total_train_time:.2f} seconds')

In [ ]:
# torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
# model = SegNet(in_channels=3, out_channels=1).to(device)
# model.load_state_dict(torch.load('model_weights.pth'))

In [ ]:
def make_predictions(
    model: torch.nn.Module,
    data: list,
    device=device
):
    model.to(device)
    model.eval()
    predictions = []
    with torch.inference_mode():
        for sample in data:
            sample = torch.unsqueeze(sample, dim=0).to(device)
            pred = model(sample)
            pred = pred.squeeze()
            predictions.append(pred)
            
    return predictions


In [ ]:
samples = random.sample(range(len(test)), 5)
predictions = make_predictions(model, [test[i][0] for i in samples])

# Show original image, ground truth depth, and predicted depth
fig, axes = plt.subplots(3, 5, figsize=(15, 9))

for i, idx in enumerate(samples):           
    image, depth_map = test[idx]
    pred = predictions[i].cpu().numpy()
    
    # Convert tensors back to numpy for plotting
    if isinstance(image, torch.Tensor):
        image = image.permute(1, 2, 0).numpy()
    if isinstance(depth_map, torch.Tensor):
        depth_map = depth_map.numpy().squeeze()
    
    # Row 1: Original images
    axes[0, i].imshow(image)
    axes[0, i].set_title(f'Image {idx}')
    axes[0, i].axis('off')
    
    # Row 2: Ground truth depth maps
    axes[1, i].imshow(depth_map, cmap='gray')
    axes[1, i].set_title(f'Ground Truth {idx}')
    axes[1, i].axis('off')
    
    # Row 3: Predicted depth maps
    axes[2, i].imshow(pred, cmap='gray')
    axes[2, i].set_title(f'Predicted {idx}')
    axes[2, i].axis('off')

plt.tight_layout()
plt.show()